# ANN Regression

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [3]:
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


### Process the data

In [4]:
df = df.drop(['RowNumber','CustomerId','Surname'], axis=1)

### Encode categorical variables

In [5]:
label_encoder_gender = LabelEncoder()
df['Gender'] = label_encoder_gender.fit_transform(df['Gender'])


In [6]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


### One Hot Encode 'Geography'

In [7]:
one_hot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = one_hot_encoder_geo.fit_transform(df[['Geography']]).toarray()
geo_encodef_df = pd.DataFrame(geo_encoded, columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))
geo_encodef_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


### Combine one-hot encoded columns with original data


In [8]:
df = pd.concat([df.drop('Geography', axis=1), geo_encodef_df], axis=1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


### Split the data into feature and target

In [9]:
X = df.drop('EstimatedSalary', axis=1)
y = df['EstimatedSalary']

### Split the data into train and test


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20, random_state=42)

### Scale the features

In [12]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Save the encoders and scaler for later use


In [13]:
with open("label_encoder_gender.pkl", "wb") as file:
    pickle.dump(label_encoder_gender, file)

with open("one_hot_encoder_geo.pkl", "wb") as file:
    pickle.dump(one_hot_encoder_geo, file)

with open("scaler.pkl", "wb") as file:
    pickle.dump(scaler, file)

## ANN Regression Problem statement

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

### Build the model

In [ ]:
model = Sequential(
    [
        Dense(64, activation='relu', input_shape = (X_train.shape[1],)),
        Dense(32, activation='relu'),
        Dense(1) # Output layer for regression, If we are not specifying anything, it will take linear regression  by default
    ]
)

### Compile the model

In [19]:
model.compile(optimizer = 'adam', loss = 'mean_absolute_error', metrics = ['mae'])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

### Tensorboard creation

In [21]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

## Setup the tensorboard
log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

### Setup early stopping

In [24]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

### Train the model

In [25]:
history = model.fit(X_train, y_train,
                    validation_data = (X_test, y_test),
                    epochs = 100,
                    callbacks = [early_stopping_callback, tensorboard_callback]
                    )

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 67112.9766 - mae: 67112.9766 - val_loss: 62492.9258 - val_mae: 62492.9258
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 61263.2188 - mae: 61263.2188 - val_loss: 54184.4609 - val_mae: 54184.4609
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 53183.8477 - mae: 53183.8477 - val_loss: 51043.1641 - val_mae: 51043.1641
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 51715.0586 - mae: 51715.0586 - val_loss: 50824.7227 - val_mae: 50824.7227
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 50266.3008 - mae: 50266.3008 - val_loss: 50346.7852 - val_mae: 50346.7852
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 51667.7422 - mae: 51667.7422 - val_loss: 50343.7227 - val_mae: 50343.7227
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 50639.2852 - mae: 50639.2852 - val_loss: 50375.6953 - val_mae: 50375.6953
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - 

In [27]:
%load_ext tensorboard


In [29]:
%tensorboard --logdir regressionlogs/fit/

Reusing TensorBoard on port 6006 (pid 13012), started 0:01:58 ago. (Use '!kill 13012' to kill it.)

### Evaluate the model on th etest data


In [31]:
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test mae", {test_mae})

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 50877.3320 - mae: 50877.3320  
Test mae {50307.3046875}


In [32]:
model.save('regression_model.h5')